# markdown.obisidian.personal.reference
> Functions for managing references in an Obsidian.md math vault

In [ ]:
#| default_exp markdown.obsidian.personal.reference

In [ ]:
#| export
import glob
import os
from os import PathLike
from pathlib import Path
import re
import shutil
from typing import Union, Optional
import warnings

from trouver.helper import (
    path_name_no_ext, alphabet_to_alphabet_group
)
from trouver.markdown.markdown.file import (
    MarkdownFile, MarkdownLineEnum
)
from trouver.markdown.markdown.heading import (
    heading_title
)
from trouver.markdown.obsidian.links import (
    ObsidianLink, LinkType, links_from_text
)
from trouver.markdown.obsidian.personal.authors import find_author_file
from trouver.markdown.obsidian.personal.index_notes import ( 
    convert_title_to_folder_name
)
from trouver.markdown.obsidian.personal.notes import (
    notes_linked_in_note
)
from trouver.markdown.obsidian.personal.note_type import (
    type_of_note, PersonalNoteTypeEnum
)
from trouver.markdown.obsidian.vault import(
    VaultNote, all_note_paths_by_name, note_path_by_name,
    NoteDoesNotExistError, NoteNotUniqueError
)

In [ ]:
import shutil
import tempfile
from unittest import mock

from fastcore.test import *

from trouver.helper import _test_directory

# Getting a reference folder

In [ ]:
#| export
def index_note_for_reference(
        vault: PathLike, # The vault in which the reference folder resides.
        reference: Union[str, Path] # - The reference. Is either - a str, in which case the reference folder will be the folder containing the (unique) note of the name `_index_{reference}.md`, - or a `Path` object (not just a pathlike!) relative to `vault`, in which case the path will be the path to the reference folder. 
        ) -> VaultNote:
    """
    Returns the index note of the specified reference in the vault.
    
    Assumes that the reference folder has an index note named
    `_index_{reference_name}.md` and this note is the unique note in the vault
    with this filename.
    
    **Raises**

    - TypeError
        - If `reference` is not a str or PathLike.
    - NoteDoesNotExistError
        - If a note of the name `_index_{reference_name}.md` does not exist
        in the vault.
    """
    if (not isinstance(reference, str)
            and not isinstance(reference, PathLike)):
        raise TypeError(
            "Expected `reference` to be a str or a PathLike, but got"
            f" {type(reference)} instead.")
    if isinstance(reference, str):
        reference_name = reference
        index_note = VaultNote(vault, name=f'_index_{reference_name}')
    elif isinstance(reference, PathLike):
        reference_name = Path(reference).name
        index_note = VaultNote(
            vault, rel_path=Path(reference) / f'_index_{reference_name}.md')
    return index_note



The `index_note_for_reference` method obtains the index note of the specified reference in the vault.

In [ ]:
vault = _test_directory() / 'test_vault_5'
index_note_1 = index_note_for_reference(vault, reference='number_theory_reference_1')
assert index_note_1.name == '_index_number_theory_reference_1'

index_note_2 = index_note_for_reference(vault, reference=Path('number_theory') / 'number_theory_reference_1')
assert index_note_2.name == '_index_number_theory_reference_1'

In [ ]:
#| export
def reference_directory(
        vault: PathLike, # The vault in which the reference folder resides.
        reference: Union[str, Path] # - The reference. Is either - a str, in which case the reference folder will be the folder containing the (unique) note of the name `_index_{reference}.md`, - or a `Path` object (not just a pathlike!) relative to `vault`, in which case the path will be the path to the reference folder. 
        ) -> Path: # Relative to `vault`.
    """
    Returns the path to the reference directory in a vault.
    
    Assumes that the reference folder has an index note named
    `_index_{reference_name}.md`, this note is the unique note in the vault
    with this filename, and the cache in the `VaultNote` class for `vault` is
    updated.

    **Raises**

    - TypeError
        - If `reference` is not a str or PathLike.
    - 
    
    """
    index_note = index_note_for_reference(vault, reference)
    if index_note.exists(update_cache=False):
        return Path(index_note.path(relative=True)).parent
    else:
        raise NoteDoesNotExistError.from_note_name(index_note.name)

The `reference_directory` method obtains the root directory for the reference:

In [ ]:
vault = _test_directory() / 'test_vault_5'
dir = reference_directory(vault, reference='number_theory_reference_1')

If the vault does not have an index note for the specified reference, then a `NoteDoesNotExistError` is raised:

In [ ]:
with ExceptionExpected(NoteDoesNotExistError):
    thing = reference_directory(vault, reference='bad_number_theory_reference_without_an_index_note')
    print(thing)

## Deleting a reference folder

In [ ]:
# export
# TODO: test delete reference and template files when reference folder does not exist.
def delete_reference_folder(
        vault: PathLike, # The vault in which the reference folder resides.
        reference: Union[str, PathLike], # The reference to delete. Is either a str, in which case the folder to delete will be the folder containing the (unique) note of the name `_index_{reference}.md`, or a path relative to `vault`. 
        verbose: bool = True,
        confirm: bool = True # If `True`, prompts the user to confirm the deletion of the folder.
        ) -> None:
    """
    Deletes a reference folder along with the associated template note
    and the reference note, both of which are outside the reference folder.
    
    Assumes that
    - the reference folder, if it exists, has an index note named
    `_index_{reference_name}.md` and this note is the unique note in the vault
    with this filename.
    - the template note, if it exists, is named `_template_{reference_name}.md`
    and is the unique note in the vault with this filename.
    - the reference note, if it exists, is named `_reference_{reference_name}.md`
    and is the unique note in the vault with this filename.
    
    If the template/reference note for the reference is not unique, then the
    deletion does not proceed. On the other hand, even if a template/reference
    note does not exist, then the deletion proceeds.

    Note that links to notes in the reference folder are preserved.
    
    **Raises**

    - FileNotFoundError
        - If the specified reference folder does not exist.
    - NoteDoesNotExistError
        - If the index note for the reference folder does not exist in the
        vault.
    - NoteNotUniqueError
        - If the index note, template note, or reference note for the reference
        folder is not unique in the vault.
    """
    try: 
        reference_path = reference_directory(vault, reference)
        absolute_path = Path(vault) / reference_path
        if verbose:
            print(f"\nIdentified reference '{reference}' in the vault '{vault}' as"
                f" the folder '{absolute_path}'...")
        reference_name = reference_path.name
    except NoteDoesNotExistError:
        reference_path = None
        reference_name = path_name_no_ext(str(reference))
    
    try:
        template_note = VaultNote(vault, name=f'_template_{reference_name}')
    except NoteDoesNotExistError:
        template_note = None
    try:
        reference_note = VaultNote(vault, name=f'_reference_{reference_name}')
    except NoteDoesNotExistError:
        reference_note = None
        
    if confirm:
        input_msg = [f"Delete"]
        if reference_path:
            input_msg = [f"\n- all contents in the folder '{absolute_path}'"]
        if template_note:
            input_msg.append(f"\n- '{template_note.path()}'")
        if reference_note:
            input_msg.append(f"\n- '{reference_note.path()}'")
        input_msg.append(f"?\n[Y/(n)]")
        command = input(''.join(input_msg))
        delete = command == 'Y'
    else:
        delete = True

    if delete:
        if verbose:
            print("Deleting...")
        if reference_path:
            shutil.rmtree(absolute_path)
            # TODO: delete the reference folder in a way that updates the cache.
        if template_note:
            template_note.delete()
        if reference_note:
            reference_note.delete()
        if verbose:
            print(f"Deleted reference.\n")
    elif verbose:
        print(f"Aborting deleting reference.\n")



The `delete_reference_folder` method deletes the reference folder itself as well as the peripheral files for the reference (the template file and reference file) which are outside of the reference folder itself.

In [ ]:
# TODO: make this example mock os.remove instead of copying the vault.
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir,
      mock.patch('__main__.input', return_value='Y') as mock_input):
    temp_vault = Path(temp_dir) / 'test_vault_5'
    shutil.copytree(_test_directory() / 'test_vault_5', temp_vault)
    
    delete_reference_folder(temp_vault, reference='number_theory_reference_1')
    assert 'number_theory_reference_1' not in os.listdir(temp_vault / 'number_theory')
    assert not VaultNote(temp_vault, name='_template_number_theory_reference_1').exists()
    assert not VaultNote(temp_vault, name='_reference_number_theory_reference_1').exists()




Identified reference 'number_theory_reference_1' in the vault 'c:\Users\hyunj\Documents\Development\Python\trouver\nbs\temp_dirne0xd3iv\test_vault_5' as the folder 'c:\Users\hyunj\Documents\Development\Python\trouver\nbs\temp_dirne0xd3iv\test_vault_5\number_theory\number_theory_reference_1'...
Deleting...
Deleted reference.

